In [1]:
!pip install transformers

     |████████████████████████████████| 389kB 2.7MB/s 
     |████████████████████████████████| 860kB 54.6MB/s 
     |████████████████████████████████| 1.0MB 40.9MB/s 
     |████████████████████████████████| 675kB 45.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=bb9dbed351040e37c8e2f0518e4357a9bbf7ec93ec6f680e42a775e03dcb009d
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609174 sha256=aee73677901e1b33c48143b76787b0a701749dad6f9a9392284cf9b16e075d95
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/TCS/sentiment-analysis/type

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/TCS/sentiment-analysis/type


In [3]:
ls

args.json  __pycache__/       test_2.ipynb  test_results.csv
data/      sentiment_2.ipynb  test.csv      utils.py
outputs/   test_002.csv       test.gsheet


In [4]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json
import pandas as pd

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from transformers import (WEIGHTS_NAME, BertConfig, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from transformers import BertModel, BertPreTrainedModel, BertForSequenceClassification
from torch import nn
from torch.nn import MSELoss, CrossEntropyLoss

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup as WarmupLinearSchedule

from utils import (convert_examples_to_features,
                        output_modes, processors, InputExample, convert_example_to_feature)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from torch import nn

In [0]:
with open('args.json', 'r') as fp:
    args = json.load(fp)

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [7]:
config = config_class.from_pretrained(args['model_name'], num_labels=4, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp4s75dlhy


INFO:transformers.file_utils:copying /tmp/tmp4s75dlhy to cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.file_utils:removing temp file /tmp/tmp4s75dlhy
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "binary",
  "hidden_act": "gelu",
  "hidden_dropout_p

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpdj9gbcdl


INFO:transformers.file_utils:copying /tmp/tmpdj9gbcdl to cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:removing temp file /tmp/tmpdj9gbcdl
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
task = args['task_name']

processor = processors[task]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [10]:
label_list

['Delivery', 'Packaging', 'Price', 'Product']

In [0]:
def test_data(test_csv, tokenizer):
    if type(test_csv) == list:
        test_strings = test_csv[:]
    else:
        test_strings = pd.read_csv(test_csv, header=None)
        test_strings = test_strings[0].to_list()
    
    output_mode = args['output_mode']
    examples = [InputExample(guid=idx, text_a=test_strings[idx], text_b=None, label='Product') for idx in range(len(test_strings))]
    features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
        
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    print("length of final dataset: ", len(dataset))
    return dataset, test_strings

In [0]:
def test(test_csv, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    softmax = nn.Softmax(dim=1)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']
    print("eval task: ", EVAL_TASK)

    eval_dataset, test_strings = test_data(test_csv, tokenizer)
    
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)
    print("len eval data: ", len(eval_dataset))


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0

    ans_preds = []
    ans_probs_neg = []
    ans_probs_pos = []

    preds = None
    probs = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None}  # XLM don't use segment_ids
#                       'labels':         batch[3]}
            outputs = model(**inputs)
            # print("outputs:----------------")
            # print(outputs)
            logits = outputs[0]
        
        # probs = softmax(logits)
        # probs = probs.detach().cpu().numpy()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            preds = np.argmax(preds, axis=1)

            probs = softmax(logits)
            probs = probs.detach().cpu().numpy()
#             out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            # print('probs shape inside else: ', probs.shape)
            preds = np.append(preds, np.argmax(logits.detach().cpu().numpy(), axis=1), axis=0)
            # print(probs)
            # print("----------------------------------")
            # print(softmax(logits).detach().cpu().numpy())
            probs = np.append(probs, softmax(logits).detach().cpu().numpy(), axis=0)
#             out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
        # probs = softmax(logits)
        # if args['output_mode'] == "classification":
        #     preds = np.argmax(preds, axis=1)
        
    # print("pred shape: ", preds.shape)
    # print("prob shape", probs.shape)
        # probs = probs.detach().cpu().numpy()
        # ans_preds.extend(probs)
    
    results = pd.DataFrame(list(zip(test_strings, preds, probs[:, 0], probs[:, 1], probs[:,2], probs[:,3])), 
               columns =['string', 'prediction', 'Delivery', 'Packaging', 'Price', 'Product'])
    
    results.to_csv('test_results.csv', index=None)
    
    return results, probs

In [13]:
model_dir = 'outputs/'
model = model_class.from_pretrained(model_dir)
model.to(device);

INFO:transformers.configuration_utils:loading configuration file outputs/config.json
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file outputs/pytorch_model.bin


In [14]:
ls

args.json  __pycache__/       test_2.ipynb  test_results.csv
data/      sentiment_2.ipynb  test.csv      utils.py
outputs/   test_002.csv       test.gsheet


In [15]:
label_list

['Delivery', 'Packaging', 'Price', 'Product']

In [0]:
test_strings = ['Product is good', 'sahi time pe mil gaya', 'compared to price, expected quality of product is good', 'except delivery, price is good']

In [16]:
preds, probs = test("test_002.csv", model, tokenizer)

eval task:  binary


100%|██████████| 11374/11374 [00:05<00:00, 1908.93it/s]
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 11374
INFO:__main__:  Batch size = 8


length of final dataset:  11374
len eval data:  11374


In [17]:
preds

,string,prediction,Delivery,Packaging,Price,Product
0,very nice product and packing also,1,0.000528,0.994646,0.002079,0.002747
1,very nice product and packing also,1,0.000528,0.994646,0.002079,0.002747
2,good,3,0.000981,0.000819,0.003794,0.994405
3,good,3,0.000981,0.000819,0.003794,0.994405
4,Loveit Mobile,3,0.029008,0.007125,0.330481,0.633386
...,...,...,...,...,...,...
11369,best quality,3,0.004036,0.004077,0.203023,0.788864
11370,Must in winter's dry skin. Leaves skin smooth.,3,0.006475,0.003954,0.272738,0.716833
11371,So light and it helps moisturize the skin and ...,3,0.005287,0.003461,0.146374,0.844879
11372,My all-time favorite. I use this from my child...,3,0.005912,0.008955,0.206780,0.778353


In [18]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/
%cd 'My Drive'
%cd TCS/sentiment-analysis

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
/gdrive/My Drive
/gdrive/My Drive/TCS/sentiment-analysis
